In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from kuma_utils.utils import sigmoid
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge

from datasets import *
from transforms import *
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [4]:
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
valid = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v18v.csv')
public748 = pd.read_csv('input/public748.csv')
iafoss_v5110 = pd.read_csv('input/predictions/val_model_v5_1_1_0.csv')

In [6]:
model_paths = [
    # Path('results/ds_09_val0'),
    Path('results/ds_09_val1'),
    Path('results/aug_04'),
    Path('results/ds_14'),
    Path('results/ds_15')
]


In [7]:
prediction_df = {}
oof_df = {}
for p in model_paths:
    if (p/'predictions_tta_wo_anomaly.npy').exists():
        prediction_df[p.stem] = sigmoid(np.load(p/'predictions_tta_wo_anomaly.npy')).reshape(-1)
    else:
        pred = sigmoid(np.load(p/'predictions_tta.npy'))
        oof = sigmoid(np.load(p/'outoffolds.npy'))
        print(pred.shape)
        if pred.shape[1] == 1:
            pred = pred.reshape(-1)
        elif pred.shape[0] == 5:
            pred = pred[0].reshape(-1)
        prediction_df[p.stem] = pred
        oof_df[p.stem] = oof.reshape(-1)
prediction_df = pd.DataFrame(prediction_df)
oof_df = pd.DataFrame(oof_df)
oof_df['iafoss_v5110'] = iafoss_v5110['target']
# prediction_df['public748'] = public748['target'].values

(7975, 1)
(7975, 1)
(7975, 1)
(7975, 1)


In [8]:
oof_df

,ds_09_val1,aug_04,ds_14,ds_15,iafoss_v5110
0,0.358545,0.388116,0.242751,0.448712,0.506348
1,0.342936,0.384529,0.321512,0.463805,0.490234
2,0.374187,0.564271,0.385565,0.490891,0.512207
3,0.443889,0.450987,0.254033,0.465469,0.558105
4,0.583250,0.542752,0.302806,0.514438,0.623535
...,...,...,...,...,...
7970,0.324713,0.382748,0.303264,0.461625,0.481689
7971,0.397259,0.385589,0.282927,0.460695,0.560059
7972,0.664132,0.778497,0.387214,0.491527,0.997559
7973,0.370571,0.393197,0.342857,0.500226,0.513184


In [10]:
model = Ridge()
X = oof_df
y = valid['target']
X_test = prediction_df
model.fit(X, y)
y_pred = model.predict(X)
print('stacking cv', roc_auc_score(y, y_pred))
print('weight', model.coef_)

stacking cv 0.7481143932186226
weight [0.14286835 0.09439106 0.06620371 0.02320658 0.62905518]


In [16]:
prediction_df

,ds_09_val1,aug_04,ds_14,ds_15
0,0.378731,0.345557,0.328906,0.506897
1,0.968598,0.984320,0.994552,0.978992
2,0.337151,0.398930,0.292780,0.458657
3,0.439851,0.539125,0.396398,0.507958
4,0.663693,0.482278,0.654054,0.599116
...,...,...,...,...
7970,0.999416,0.998426,0.999989,0.830332
7971,0.486552,0.544869,0.434396,0.558710
7972,0.262954,0.328540,0.262360,0.460538
7973,0.448427,0.584629,0.329145,0.600259


In [13]:
# test['target'] = prediction_df['aug_04'] * 0.2 + prediction_df['ds_05_aug2'] * 0.2 + prediction_df['public748'] * 0.6
# test['target'] = prediction_df['ds_09'] * 0.33 + prediction_df['ds_13'] * 0.33 + prediction_df['public748'] * 0.33
test['target'] = prediction_df['ds_15']

In [14]:
test

,id,target
0,00054c878,0.506897
1,0007285a3,0.978992
2,00076c5a6,0.458657
3,001349290,0.507958
4,001a52e92,0.599116
...,...,...
7970,ffbce04ef,0.830332
7971,ffc2d976b,0.558710
7972,ffc905909,0.460538
7973,ffe276f3e,0.600259


In [15]:
# test.to_csv('results/blend_submission.csv', index=False)
test.to_csv(Path('results/ds_15')/'submission.csv', index=False)